In [ ]:
from catboost.datasets import amazon

In [2]:
(train_df, test_df) = amazon()


In [3]:
train_df.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [4]:
y = train_df.ACTION
X = train_df.drop("ACTION", axis=1)

In [5]:
cat_features = list(range(0, X.shape[1]))
print(cat_features)
print("Labels: {}".format(set(y)))
print("Zero count = {}, One count = {}".format(len(y) - sum(y), sum(y)))

[0, 1, 2, 3, 4, 5, 6, 7, 8]
Labels: {0, 1}
Zero count = 1897, One count = 30872


# Training the first model

In [6]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=100)
model.fit(X, y, cat_features=cat_features, verbose=10)

Learning rate set to 0.377604
0:	learn: 0.4528598	total: 94.2ms	remaining: 9.33s
10:	learn: 0.1744186	total: 250ms	remaining: 2.02s
20:	learn: 0.1676119	total: 347ms	remaining: 1.31s
30:	learn: 0.1652446	total: 423ms	remaining: 942ms
40:	learn: 0.1633644	total: 529ms	remaining: 761ms
50:	learn: 0.1621892	total: 646ms	remaining: 621ms
60:	learn: 0.1609164	total: 789ms	remaining: 504ms
70:	learn: 0.1594572	total: 900ms	remaining: 368ms
80:	learn: 0.1585876	total: 1.01s	remaining: 237ms
90:	learn: 0.1573593	total: 1.17s	remaining: 115ms
99:	learn: 0.1566977	total: 1.3s	remaining: 0us


In [7]:
model.predict_proba(X)

array([[0.00977075, 0.99022925],
       [0.01005186, 0.98994814],
       [0.05793081, 0.94206919],
       ...,
       [0.01181209, 0.98818791],
       [0.18912026, 0.81087974],
       [0.02352545, 0.97647455]])

In [8]:
from catboost import Pool
pool = Pool(data=X, label=y, cat_features=cat_features)

In [9]:
from sklearn.model_selection import train_test_split

data = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_validation, y_train, y_validation = data

train_pool = Pool(
    data=X_train,
    label=y_train,
    cat_features=cat_features
)

validation_pool = Pool(
    data = X_validation,
    label = y_validation,
    cat_features=cat_features
)



In [10]:
from catboost import CatBoostClassifier

# selecting the objective function
`Logloss` for binary target
`CrossEntropy` for probability in target

In [11]:
model = CatBoostClassifier(
    iterations=5,
    learning_rate = 0.1,
    # loss_function='CrossEntropy'
)
model.fit(train_pool, eval_set=validation_pool, verbose=False)

print("Model is fitted: {}".format(model.is_fitted()))
print("Model params:\n{}".format(model.get_params()))

Model is fitted: True
Model params:
{'iterations': 5, 'learning_rate': 0.1}


In [14]:
model = CatBoostClassifier(
    iterations=15,
)
model.fit(train_pool, eval_set=validation_pool)


Learning rate set to 0.441257
0:	learn: 0.4226231	test: 0.4217069	best: 0.4217069 (0)	total: 6.41ms	remaining: 89.7ms
1:	learn: 0.3157972	test: 0.3136469	best: 0.3136469 (1)	total: 15.8ms	remaining: 102ms
2:	learn: 0.2631196	test: 0.2603395	best: 0.2603395 (2)	total: 21ms	remaining: 84.1ms
3:	learn: 0.2334650	test: 0.2294580	best: 0.2294580 (3)	total: 26ms	remaining: 71.5ms
4:	learn: 0.2077060	test: 0.2017327	best: 0.2017327 (4)	total: 30.3ms	remaining: 60.7ms
5:	learn: 0.1961364	test: 0.1883112	best: 0.1883112 (5)	total: 34.1ms	remaining: 51.2ms
6:	learn: 0.1879266	test: 0.1794018	best: 0.1794018 (6)	total: 39.1ms	remaining: 44.7ms
7:	learn: 0.1841218	test: 0.1743149	best: 0.1743149 (7)	total: 41.4ms	remaining: 36.3ms
8:	learn: 0.1814626	test: 0.1698731	best: 0.1698731 (8)	total: 48.1ms	remaining: 32.1ms
9:	learn: 0.1785403	test: 0.1650335	best: 0.1650335 (9)	total: 51.4ms	remaining: 25.7ms
10:	learn: 0.1771678	test: 0.1634002	best: 0.1634002 (10)	total: 55.7ms	remaining: 20.2ms
11:	l

In [ ]:
# Metrics calculation and graph plotting
